# Comparison of Model Performance

All models are compared based on their forecast performannce on the period from 2024-01-23 to 2025-02-28. (including two ends).

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import t
from scipy.stats import norm
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

- Forecast Error Metrics (MAE, RMSE, MASE)

- Diebold-Mariano Test (for forecast superiority)

- Visual Comparison (actual vs. predicted volatility)

One of the considerations was choosing a recursive estimation scheme over a rolling scheme in forecast estimation. While the latter is better suited for purposes like forecast comparison using DM test (which assumes that the differential is covariance stationary), the recursive estimation scheme is still favoured for practical reasons. This is because by expanding the training set timeframe, it uses more and more data to make later forecasts - parameter uncertainty is reduced, and forecast error may become less variable. Also, parameter estimation uncertainty typically constitutes a tiny fraction of forecast error variance, so reducing it would not noticeably violate stationarity. 

In [16]:
# Diebold Mariano (1995) EPA Test
def diebold_mariano(actual, forecast1, forecast2, h=3):
    """
    DM test for h-step-ahead forecasts.
    H0: Forecasts have equal accuracy.
    HA: Forecast 1 is more accurate than Forecast 2.
    """
    # Forecast errors
    e1 = actual - forecast1
    e2 = actual - forecast2
    
    # Loss differential (squared errors)
    d = e1**2 - e2**2
    
    # DM statistic (with Newey-West adjustment for autocorrelation)
    n = len(d)
    d_mean = np.mean(d)
    d_var = np.var(d, ddof=1)  # HAC adjustment not shown here (see next section)
    dm_stat = d_mean / np.sqrt(d_var / n)
    
    # Critical value (standard normal for large n)
    p_value = 2 * norm.sf(np.abs(dm_stat))  # Two-tailed test
    
    return dm_stat, p_value

In [25]:
results = pd.DataFrame(columns=['Model', 'Metric', '1D', '3D', '7D', '30D'])

for model in ['GARCH(1,0,1)', 'GARCH-SVM']:
    maes = []
    rmses = []
    mapes = []
    r2s = []

    for h in [1, 3, 7, 30]:
        # Load the predictions
        predictions = pd.read_csv(f'../res/{model}_{h}D.csv')['Predicted']
        actual = pd.read_csv(f'../res/lnRV_test_{h}D.csv')['lnRV']
        
        # Calculate metrics and append to lists
        mae = mean_absolute_error(actual, predictions)
        rmse = np.sqrt(mean_squared_error(actual, predictions))
        mape = mean_absolute_percentage_error(actual, predictions) * 100
        r2 = r2_score(actual, predictions)

        maes.append(mae)
        rmses.append(rmse)
        mapes.append(mape)
        r2s.append(r2)

    # Append results to DataFrame
    results = pd.concat([results, pd.DataFrame({
        'Model': [model] * 4,
        'Metric': ['MAE', 'RMSE', 'MAPE', 'R-squared'],
        '1D': [maes[0], rmses[0], mapes[0], r2s[0]],
        '3D': [maes[1], rmses[1], mapes[1], r2s[1]],
        '7D': [maes[2], rmses[2], mapes[2], r2s[2]],
        '30D': [maes[3], rmses[3], mapes[3], r2s[3]],
    })], ignore_index=True)

results

C:\Users\65835\AppData\Local\Temp\ipykernel_15136\1011208853.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame({


,Model,Metric,1D,3D,7D,30D
0,"GARCH(1,0,1)",MAE,0.464673,0.499103,0.585017,0.883290
1,"GARCH(1,0,1)",RMSE,0.618827,0.652981,0.727710,1.018471
2,"GARCH(1,0,1)",MAPE,8.344001,8.823486,10.148567,14.930754
3,"GARCH(1,0,1)",R-squared,0.426290,0.349858,0.198842,-0.539389
4,GARCH-SVM,MAE,0.405708,0.393386,0.487733,0.592328
5,GARCH-SVM,RMSE,0.567377,0.542832,0.649414,0.795584
6,GARCH-SVM,MAPE,7.445700,7.153618,8.795893,10.803316
7,GARCH-SVM,R-squared,0.517722,0.550698,0.361964,0.060660


In [ ]:
maes = []
rmses = []
mapes = []
r2s = []

for h in [1, 3, 7, 30]:
    # Load the predictions
    predictions = pd.read_csv(f'../res/'HAR(1, 7, 30)'_{h}D.csv')['Pred']
    actual = pd.read_csv(f'../res/lnRV_test_{h}D.csv')['lnRV']
    
    # Calculate metrics and append to lists
    mae = mean_absolute_error(actual, predictions)
    rmse = np.sqrt(mean_squared_error(actual, predictions))
    mape = mean_absolute_percentage_error(actual, predictions) * 100
    r2 = r2_score(actual, predictions)

    maes.append(mae)
    rmses.append(rmse)
    mapes.append(mape)
    r2s.append(r2)

# Append results to DataFrame
results = pd.concat([results, pd.DataFrame({
    'Model': [model] * 4,
    'Metric': ['MAE', 'RMSE', 'MAPE', 'R-squared'],
    '1D': [maes[0], rmses[0], mapes[0], r2s[0]],
    '3D': [maes[1], rmses[1], mapes[1], r2s[1]],
    '7D': [maes[2], rmses[2], mapes[2], r2s[2]],
    '30D': [maes[3], rmses[3], mapes[3], r2s[3]],
})], ignore_index=True)


In [ ]:
for h in [1,3,7,30]:
    # load data
    garch = pd.read_csv(f'../res/GARCH(1,0,1)_{h}D.csv')['Predicted']
    har = pd.read_csv(f'../res/HAR(1, 7, 30)_{h}D.csv')['Pred']
    garch_svm = pd.read_csv(f'../res/GARCH-SVM_{h}D.csv')['Predicted']
    actual = pd.read_csv(f'../res/lnRV_test_{h}D.csv')['lnRV']

    # Mean Absolute Error (MASE)
    mae_garch = mean_absolute_error(actual, garch)
    mae_har = mean_absolute_error(actual, har)
    mae_garch_svm = mean_absolute_error(actual, garch_svm)

    # Mean Absolute Error (MASE)
    mae_garch = mean_absolute_error(actual, garch)
    mae_har = mean_absolute_error(actual, har)
    mae_garch_svm = mean_absolute_error(actual, garch_svm)

    # Root Mean Squared Error (RMSE)
    rmse_garch = np.sqrt(mean_squared_error(actual, garch))
    rmse_har = np.sqrt(mean_squared_error(actual, har))
    rmse_garch_svm = np.sqrt(mean_squared_error(actual, garch_svm))

    # Mean Absolute Percentage Error (MAPE)
    mape_garch = mean_absolute_percentage_error(actual, garch) * 100
    mape_har = mean_absolute_percentage_error(actual, har) * 100
    mape_garch_svm = mean_absolute_percentage_error(actual, garch_svm) * 100

    # Mean Absolute Scaled Error (MASE) - Lagged realized vol as benchmark
    naive_forecast = np.roll(actual, h)
    naive_forecast[:h] = np.nan  # Set the first h values to NaN
    mase_garch = mean_absolute_error(actual[h:], garch[h:]) / mean_absolute_error(actual[h:], naive_forecast[h:])
    mase_har = mean_absolute_error(actual[h:], har[h:]) / mean_absolute_error(actual[h:], naive_forecast[h:])
    mase_garch_svm = mean_absolute_error(actual[h:], garch_svm[h:]) / mean_absolute_error(actual[h:], naive_forecast[h:])

    # R-squared
    r2_garch = r2_score(actual, garch)
    r2_har = r2_score(actual, har)
    r2_garch_svm = r2_score(actual, garch_svm)

    # Diebold-Mariano test
    dm_stat_garch_har, p_value_garch_har = diebold_mariano(actual, garch, har)
    dm_stat_garch_svm, p_value_garch_svm = diebold_mariano(actual, garch, garch_svm)
    dm_stat_har_svm, p_value_har_svm = diebold_mariano(actual, har, garch_svm)

    # save results to csv
    results = pd.DataFrame({
        'Metric': ['MAE', 'RMSE', 'MAPE', 'MASE', 'R-squared'],
        'GARCH': [mae_garch, rmse_garch, mape_garch, mase_garch, r2_garch],
        'HAR': [mae_har, rmse_har, mape_har, mase_har, r2_har],
        'GARCH-SVM': [mae_garch_svm, rmse_garch_svm, mape_garch_svm, mase_garch_svm, r2_garch_svm],
    })
    results.to_csv(f'../eval/metrics_{h}D.csv', index=False)

    # save DM test results to csv
    dm_results = pd.DataFrame({
        'Comparison': ['GARCH vs HAR', 'GARCH vs GARCH-SVM', 'HAR vs GARCH-SVM'],
        'DM Statistic': [dm_stat_garch_har, dm_stat_garch_svm, dm_stat_har_svm],
        'p-value': [p_value_garch_har, p_value_garch_svm, p_value_har_svm]
    })
    dm_results.to_csv(f'../eval/DM_test_{h}D.csv', index=False)

In [22]:
# print results
print("Metrics for GARCH, HAR, and GARCH-SVM models:")
print("------------------------------------------------")
print("MAE:")
print(f"GARCH: {mae_garch:.4f}, HAR: {mae_har:.4f}, GARCH-SVM: {mae_garch_svm:.4f}")
print("RMSE:")
print(f"GARCH: {rmse_garch:.4f}, HAR: {rmse_har:.4f}, GARCH-SVM: {rmse_garch_svm:.4f}")
print("MAPE:")
print(f"GARCH: {mape_garch:.4f}%, HAR: {mape_har:.4f}%, GARCH-SVM: {mape_garch_svm:.4f}%")
print("MASE:")
print(f"GARCH: {mase_garch:.4f}, HAR: {mase_har:.4f}, GARCH-SVM: {mase_garch_svm:.4f}")
print("R-squared:")
print(f"GARCH: {r2_garch:.4f}, HAR: {r2_har:.4f}, GARCH-SVM: {r2_garch_svm:.4f}")
print("\nDiebold-Mariano Test Results:")
print("------------------------------------------------")
print(f"GARCH vs HAR: DM Statistic = {dm_stat_garch_har:.4f}, p-value = {p_value_garch_har:.4f}")
print(f"GARCH vs GARCH-SVM: DM Statistic = {dm_stat_garch_svm:.4f}, p-value = {p_value_garch_svm:.4f}")
print(f"HAR vs GARCH-SVM: DM Statistic = {dm_stat_har_svm:.4f}, p-value = {p_value_har_svm:.4f}")

Metrics for GARCH, HAR, and GARCH-SVM models:
------------------------------------------------
MAE:
GARCH: 0.8833, HAR: 0.3848, GARCH-SVM: 0.5923
RMSE:
GARCH: 1.0185, HAR: 0.5304, GARCH-SVM: 0.7956
MAPE:
GARCH: 14.9308%, HAR: 6.9800%, GARCH-SVM: 10.8033%
MASE:
GARCH: 1.0312, HAR: 0.4703, GARCH-SVM: 0.6903
R-squared:
GARCH: -0.5394, HAR: 0.5825, GARCH-SVM: 0.0607

Diebold-Mariano Test Results:
------------------------------------------------
GARCH vs HAR: DM Statistic = 13.4567, p-value = 0.0000
GARCH vs GARCH-SVM: DM Statistic = 7.6188, p-value = 0.0000
HAR vs GARCH-SVM: DM Statistic = -7.7838, p-value = 0.0000
